In [ ]:
import geopandas as gpd

In [ ]:
# ok, let's start fiddling
bedrock_data = gpd.read_file('BedrockP.gpkg')
bedrock_data.plot(
    column="era", categorical=True, legend=True, figsize=(20,20), cmap='gist_earth')

In [ ]:
# So, what do I want to do to this?
# I want to bluntly solve the problem first, then improve the plotting, then think about UI.

# Classify polygons as 'serpentinite', 'granodiorite', 'ultramafic'
# Do I need to look at some of fuzzy match edge cases?
from functools import partial
from rapidfuzz.fuzz import partial_ratio

for match_threshold in [80, 100]:
    search_terms = ['serpentinite', 'granodiorite', 'ultramafic']
    fuzzy_matches_already_reported = []

    def fuzzy_match(cell, search_term, match_threshold) -> bool:
        # case insensitive fuzzy match
        if isinstance(cell, str):
            match_pct = partial_ratio(search_term, cell, processor=lambda x: x.lower())
            if match_pct >= match_threshold:
                if match_pct < 100:
                    print(f'Fuzzy match for "{search_term}" found at {match_pct} pct: "{cell}"')
                return True
        return False

    for term in search_terms:
        key = f'is_{term}_{match_threshold}pct_match'
        bedrock_data[key] = bedrock_data.map(partial(fuzzy_match, search_term=term, match_threshold=match_threshold)).any(axis=1)
        print(f'{bedrock_data[key].sum()} of {len(bedrock_data)} polygons are {term}, fuzzy match {match_threshold} pct')

        col_match = bedrock_data.map(partial(fuzzy_match, search_term=term, match_threshold=match_threshold)).any()
        relevant_columns = col_match[col_match].index.tolist()
        print(f'relevant columns are {relevant_columns}')

In [ ]:
# Do the polygons in the data overlap or not?
bedrock_data.sindex

# Create an overlap matrix
overlap_matrix = bedrock_data.geometry.apply(lambda g: bedrock_data.sindex.query(g, predicate='overlaps')).apply(
    lambda idxs: bedrock_data.index.isin(idxs)
)

# Assert nothing overlaps
assert overlap_matrix.sum().sum() == 0

# Phew, they don't overlap. that's one job made easier.

In [ ]:
# ok, if the polygons don't overlap, we should be able to dilate the edges of the relevant rock types
# and intersect those dilations to gradually plot out the heatmap.

# We can't directly save polygons here though, as we need to provide a result for every single point.
# If we use purely polygons, we'll end up with floating point errors introducing slivers between them that are not properly categorised.
# We need to rasterise to guarantee that we'll have a valid value for every single query point.

from affine import Affine
from rasterio.features import rasterize
from shapely import intersection
import numpy as np

max_rock_separation_m = 10e3
sepmap_resolution_m = 100
separation_levels_m = np.arange(0, max_rock_separation_m, 0.5e3)

rock_type_a_geom = bedrock_data.loc[bedrock_data['is_granodiorite_80pct_match'],'geometry'].union_all()
rock_type_b_geom = bedrock_data.loc[bedrock_data['is_serpentinite_80pct_match'] | bedrock_data['is_ultramafic_80pct_match'],'geometry'].union_all()

# calculate the polygon covering the max separation distance
max_separation_poly = intersection(
    rock_type_a_geom.buffer(max_rock_separation_m/2),
    rock_type_b_geom.buffer(max_rock_separation_m/2)
)

# use this polygon to establish bounds for the separation map
sepmap_minx, sepmap_miny, sepmap_maxx, sepmap_maxy = max_separation_poly.bounds

# add a small buffer for ease of use
sepmap_minx -= 2 * max_rock_separation_m
sepmap_miny -= 2 * max_rock_separation_m
sepmap_maxx += 2 * max_rock_separation_m
sepmap_maxy += 2 * max_rock_separation_m

# Calculate heatmap dimensions and projection
sepmap_width_pixels = int(np.ceil((sepmap_maxx - sepmap_minx) / sepmap_resolution_m))
sepmap_height_pixels = int(np.ceil((sepmap_maxy - sepmap_miny) / sepmap_resolution_m))
print(f'rock separation map is {sepmap_width_pixels}x{sepmap_height_pixels} pixels')
raster_transform = Affine(sepmap_resolution_m, 0, sepmap_minx, 0, -sepmap_resolution_m, sepmap_maxy)

# Create empty separation map - note that this is just a numpy array!
sepmap = np.full((sepmap_height_pixels, sepmap_width_pixels), fill_value=np.nan, dtype=np.float32)

# starting from the largest separation value (lowest probability) and working down,
# calculate the appropriate polygon using buffering, then assign all points within that buffer a lower separation value / higher probability
for d in sorted(separation_levels_m, reverse=True):
    is_rock_types_within_distance_poly = intersection(
            rock_type_a_geom.buffer(d/2),
            rock_type_b_geom.buffer(d/2)
    )

    is_rock_types_within_distance_raster = rasterize(
        [(is_rock_types_within_distance_poly, 1)],
        out_shape=sepmap.shape,
        transform=raster_transform,
        dtype="uint8"
    )

    # update pixel values covered by this polygon
    sepmap[is_rock_types_within_distance_raster == 1] = d

# this guarantees that all pixels have an appropriate value or nan, and avoids having to calculate distances for every single pixel
# There's a slight loss of accuracy compared to calculating fine distances for every pixel, but the extra speed here allows us
# to up the resolution to make up for that.

In [ ]:
# let's visualise the polygon maths as a sense check
from rasterio.transform import xy
import matplotlib.pyplot as plt
import contextily as ctx

# plot type a
ax = gpd.GeoSeries(rock_type_a_geom, crs=bedrock_data.crs).plot(figsize=(20,20), color='blue', alpha=0.7)
fig = plt.gcf()

# plot type b
gpd.GeoSeries(rock_type_b_geom, crs=bedrock_data.crs).plot(ax=ax, color='red', alpha=0.7)

# plot max overlap polygon
gpd.GeoSeries(max_separation_poly, crs=bedrock_data.crs).plot(ax=ax, color='green', alpha=0.5)

# add a basemap
ctx.add_basemap(ax, source=ctx.providers.CartoDB.Positron, crs=bedrock_data.crs)

ax.set_title("Polygon sense check", fontsize=14)
fig.tight_layout()
ax.grid()
ax.set_xlabel(f' EPSG:{bedrock_data.crs.to_epsg()} X (m)')
ax.set_ylabel(f' EPSG:{bedrock_data.crs.to_epsg()} Y (m)')
plt.show()

In [ ]:
# Next, let's plot the heatmap to produce a useable final image

# convert from separation of rock types to probability
def separation_distance_to_likelihood(dist: np.ndarray, max_dist: float, steepness: float = np.nan) -> np.ndarray:
    # use a bounded sigmoid.
    # work out a sensible steepness if not provided:
    if np.isnan(steepness):
        steepness = 2 / max_dist * np.log((1 / 1e-3)-1)
    output = np.zeros(dist.shape)
    output[dist <= max_dist] = 1 / (1 + np.exp(steepness*(dist[dist <= max_dist].ravel()-max_dist/2)))
    return output.reshape(dist.shape)

heatmap = separation_distance_to_likelihood(sepmap, max_dist=10e3)

sepmap_extent = (sepmap_minx, sepmap_maxx, sepmap_miny, sepmap_maxy)  # bounds in CRS coordinates

fig, ax = plt.subplots(figsize=(20, 20))

ax.set_xlim(sepmap_minx, sepmap_maxx)
ax.set_ylim(sepmap_miny, sepmap_maxy)

# add a basemap, setting the zoom manually so the result is a bit nicer
ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik, crs=bedrock_data.crs, zoom=9)

cmap = plt.get_cmap("plasma_r")

# set the alpha channel to the value of the heatmap, so that low probabilities fade to the background image
heatmap_for_plotting = cmap(heatmap)
heatmap_for_plotting[..., -1] = heatmap

# overplot heatmap
img = ax.imshow(
    heatmap_for_plotting,
    extent=sepmap_extent,
    origin="upper",
    cmap=cmap,
    alpha=0.7
)

ax.set_title("Likelihood of Cobalt deposit i.v.o Squamish, Canada", fontsize=14)
fig.tight_layout()
ax.grid()
ax.set_xlabel(f' EPSG:{bedrock_data.crs.to_epsg()} X (m)')
ax.set_ylabel(f' EPSG:{bedrock_data.crs.to_epsg()} Y (m)')
cbar = fig.colorbar(img, ax=ax, label="Likelihood of Cobalt deposit")
plt.show()

In [ ]:
# How do I enable the analyst to calculate the exact probability at a specific point?
import rasterio
from rasterio.transform import rowcol

# start by saving the raster to a file

# Save to GeoTIFF
with rasterio.open(
    "cobalt_ivo_squamish.geotiff",
    'w',
    driver='GTiff',
    height=heatmap.shape[0],
    width=heatmap.shape[1],
    count=1,
    dtype=heatmap.dtype,
    crs=bedrock_data.crs,
    transform=raster_transform
) as dst:
    dst.write(heatmap, 1)


# Example query point (in same CRS as raster)
query_x, query_y = 550e3, 5.6e6

with rasterio.open("cobalt_ivo_squamish.geotiff") as src:
    data = src.read(1)  # first band
    transform = src.transform

    # Find nearest pixel row/col to query point
    row, col = rowcol(transform, query_x, query_y)

    # Clamp to raster bounds
    row = np.clip(row, 0, src.height - 1)
    col = np.clip(col, 0, src.width - 1)

    # Return nearest neighbour value - our raster resolution is small enough for this to be adequate.
    value = data[row, col]

print(f"Coblat likelihood at ({query_x}, {query_y}) is {value:.2f}")


## UI/structure considerations
ok, let's list all of the sensible inputs and then think about how to design a UI for them, if indeed I do.
- select file to load
- specify probability curve
  - input function and visualise it
  - piecewise function?
- specify fuzzy match percent for rock labels
  - visualise examples from data for fuzzy matches
- select map background
- zoom in/out
- left/right sweeper?
- specify heatmap accuracy
  - number of levels
  - raster resolution
  - number of edges in corner polygons
- save heatmap
  - as shapefile
  - as geopackage
  - as png

## Algorithm speed
- option to simplify polygons before calculation?

## Technologies demonstrated
- I'm not using PostGRES right now.